# User-based recommendation on user clusters (based on likes)

## Import

In [1]:
import pandas as pd
import ast

In [109]:
data_profiles = pd.read_csv('../data/profiles_clean.csv')
data_reviews = pd.read_csv('../data/reviews_clean.csv')

display(data_profiles.head(2))
display(data_reviews.head(2))

,profile,gender,birthday,favorites_anime,link,age
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche,26.0
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans,20.0


,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117


## Preprocessing

In [110]:
data_profiles['favorites_anime'] = data_profiles['favorites_anime'].apply(ast.literal_eval)

df_als_favorite = data_profiles[["profile", "favorites_anime"]].copy().explode('favorites_anime')
df_als_favorite = df_als_favorite.dropna(subset=["favorites_anime"])
df_als_favorite["favorites_anime"] = df_als_favorite["favorites_anime"].astype('int64')
df_als_favorite["is_favorite"] = 1

display(df_als_favorite.head(2))

,profile,favorites_anime,is_favorite
0,DesolatePsyche,33352,1
0,DesolatePsyche,25013,1


In [111]:
df_als_score = data_reviews[["profile", "anime_uid", "score"]].copy()

display(df_als_score.head(2))

,profile,anime_uid,score
0,DesolatePsyche,34096,8
1,baekbeans,34599,10


In [112]:
df_als_favorite.to_csv("../data/als_is_favorite.csv", index=False)
df_als_score.to_csv("../data/als_score.csv", index=False)

## Spark

For now, this part have to be executed in a ad-hoc Jupyter environment with PySpark, following those steps (disclaimer : you need to install and configure PySpark first) :


```shell
pyspark --name anime-recommendation-engine --driver-java-options -Djava.security.manager=allow
```

```python
sc = SparkSession.builder.getOrCreate()
```

[Medium article](https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3)


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS

In [5]:
sc = SparkSession.builder.getOrCreate()

In [7]:
df_als_favorite = pd.read_csv('../data/als_is_favorite.csv')
df_als_score = pd.read_csv('../data/als_score.csv')

display(df_als_favorite.head(2)) # (216695, 3)
display(df_als_score.head(2)) # (130519, 3)

,profile,favorites_anime,is_favorite
0,DesolatePsyche,33352,1
1,DesolatePsyche,25013,1


,profile,anime_uid,score
0,DesolatePsyche,34096,8
1,baekbeans,34599,10


In [37]:
df_mean_favorite = df_als_favorite.groupby("profile")["is_favorite"].count().reset_index()
df_mean_favorite["is_favorite"].mean()

5.7842404505779035

In [39]:
df_mean_score = df_als_score.groupby("profile")["score"].count().reset_index()
df_mean_score["score"].mean()

2.7256760989871567

### Is favorite

In [51]:
df_sc_is_favorite = sc.createDataFrame(df_als_favorite)
df_sc_is_favorite.show()

+--------------+---------------+-----------+
|       profile|favorites_anime|is_favorite|
+--------------+---------------+-----------+
|DesolatePsyche|          33352|          1|
|DesolatePsyche|          25013|          1|
|DesolatePsyche|           5530|          1|
|DesolatePsyche|          33674|          1|
|DesolatePsyche|           1482|          1|
|DesolatePsyche|            269|          1|
|DesolatePsyche|          18245|          1|
|DesolatePsyche|           2904|          1|
|DesolatePsyche|          27899|          1|
|DesolatePsyche|          17074|          1|
|DesolatePsyche|          12291|          1|
|DesolatePsyche|            226|          1|
|DesolatePsyche|          28851|          1|
|DesolatePsyche|           8525|          1|
|DesolatePsyche|           6594|          1|
|DesolatePsyche|           4981|          1|
|DesolatePsyche|           1698|          1|
|DesolatePsyche|            457|          1|
|DesolatePsyche|            235|          1|
|DesolateP

In [53]:
indexer = StringIndexer(inputCol="profile", outputCol="profile_index")

transformed = indexer.fit(df_sc_is_favorite).transform(df_sc_is_favorite)
transformed.show()

+--------------+---------------+-----------+-------------+
|       profile|favorites_anime|is_favorite|profile_index|
+--------------+---------------+-----------+-------------+
|DesolatePsyche|          33352|          1|         20.0|
|DesolatePsyche|          25013|          1|         20.0|
|DesolatePsyche|           5530|          1|         20.0|
|DesolatePsyche|          33674|          1|         20.0|
|DesolatePsyche|           1482|          1|         20.0|
|DesolatePsyche|            269|          1|         20.0|
|DesolatePsyche|          18245|          1|         20.0|
|DesolatePsyche|           2904|          1|         20.0|
|DesolatePsyche|          27899|          1|         20.0|
|DesolatePsyche|          17074|          1|         20.0|
|DesolatePsyche|          12291|          1|         20.0|
|DesolatePsyche|            226|          1|         20.0|
|DesolatePsyche|          28851|          1|         20.0|
|DesolatePsyche|           8525|          1|         20.

25/05/28 11:51:17 WARN DAGScheduler: Broadcasting large task binary with size 1224.1 KiB


In [55]:
(training,test)=transformed.randomSplit([0.8, 0.2])

In [57]:
# There's a mean of 5.78 liked animes by profile, so we decide to use
# rank = 5
# maxIter = 10
# regParam = 0.1
als = ALS(rank=5,
          userCol="profile_index", itemCol="favorites_anime",
          ratingCol="is_favorite", coldStartStrategy="drop", nonnegative=True)

model = als.fit(training)

25/05/28 11:51:23 WARN DAGScheduler: Broadcasting large task binary with size 1253.0 KiB
25/05/28 11:51:23 WARN DAGScheduler: Broadcasting large task binary with size 1255.3 KiB
25/05/28 11:51:23 WARN DAGScheduler: Broadcasting large task binary with size 1256.8 KiB
25/05/28 11:51:23 WARN DAGScheduler: Broadcasting large task binary with size 1258.1 KiB
25/05/28 11:51:23 WARN DAGScheduler: Broadcasting large task binary with size 1257.1 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1258.4 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1259.2 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1262.3 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1263.7 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1265.1 KiB
25/05/28 11:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1266.4 KiB
25/05/28 11:51:24 WAR

In [59]:
evaluator = RegressionEvaluator(metricName="rmse",labelCol="is_favorite",predictionCol="prediction")

predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

print("RMSE="+str(rmse))

predictions.show()

25/05/28 11:51:30 WARN DAGScheduler: Broadcasting large task binary with size 1235.3 KiB
25/05/28 11:51:30 WARN DAGScheduler: Broadcasting large task binary with size 1297.3 KiB
25/05/28 11:51:30 WARN DAGScheduler: Broadcasting large task binary with size 1295.9 KiB
25/05/28 11:51:31 WARN DAGScheduler: Broadcasting large task binary with size 1343.5 KiB


RMSE=0.09981547241451366


25/05/28 11:51:31 WARN DAGScheduler: Broadcasting large task binary with size 1235.4 KiB
25/05/28 11:51:31 WARN DAGScheduler: Broadcasting large task binary with size 1297.3 KiB
25/05/28 11:51:31 WARN DAGScheduler: Broadcasting large task binary with size 1295.9 KiB


+----------------+---------------+-----------+-------------+----------+
|         profile|favorites_anime|is_favorite|profile_index|prediction|
+----------------+---------------+-----------+-------------+----------+
|     spartan_073|          30654|          1|      14801.0|  0.900804|
|          Ilieas|          17389|          1|       2255.0| 0.9007878|
|      Crusader_8|           1088|          1|      10779.0| 0.9007751|
|         Mac_kun|          30654|          1|      11196.0| 0.9007956|
|     ChoocoboYao|          30654|          1|       1137.0| 0.9007934|
|          Wuster|           1088|          1|      21708.0|0.90077496|
|        pandapow|           1088|          1|      24847.0| 0.9007811|
|Bishounen_Hunter|           1088|          1|      15786.0|0.90078074|
|         MrScout|          30654|          1|      33160.0| 0.9007913|
|  Akuma-Shitsuji|           5300|          1|       9074.0| 0.9007893|
|          sterl1|           1088|          1|       7097.0|0.90

In [79]:
recs=model.recommendForAllUsers(5).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['profile_index'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna() 
nrecs=nrecs.sort_values('profile_index')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['profile_index']], axis = 1)
nrecs.columns = [        
        'favorites_anime',
        'is_favorite',
        'UserID_index'      
     ]
md=transformed.select(transformed['profile'],transformed['profile_index'],transformed['favorites_anime'])
md=md.toPandas()
dict1 =dict(zip(md['profile_index'],md['profile']))
nrecs['profile']=nrecs['UserID_index'].map(dict1)
nrecs=nrecs.sort_values('profile')
nrecs.reset_index(drop=True, inplace=True)
new=nrecs[['profile','favorites_anime','is_favorite']]
new['recommendations'] = list(zip(new.favorites_anime, new.is_favorite))
res=new[['profile','recommendations']]  
df_favorites_based_reco=res['recommendations'].groupby([res.profile]).apply(list).reset_index()
print(df_favorites_based_reco.head())

25/05/28 11:55:16 WARN DAGScheduler: Broadcasting large task binary with size 1344.6 KiB
25/05/28 11:55:18 WARN DAGScheduler: Broadcasting large task binary with size 1338.5 KiB
25/05/28 11:55:25 WARN DAGScheduler: Broadcasting large task binary with size 1223.5 KiB


            profile                                    recommendations
0  -----noname-----  [(23703.0, 0.9009409546852112), (40256.0, 0.90...
1   ---SnowFlake---  [(11859.0, 0.9010751247406006), (23703.0, 0.90...
2          --Mizu--  [(11859.0, 0.9010722041130066), (23703.0, 0.90...
3     --Sunclaudius  [(7103.0, 0.9009591341018677), (23703.0, 0.900...
4   --animeislife--  [(40256.0, 0.900961697101593), (7103.0, 0.9009...


In [ ]:
# TODO : export to Pandas DataFrame + export to.csv

In [81]:
df_favorites_based_reco.to_csv("../data/als_is_favorite_based_reco.csv", index=False)

## Score

In [83]:
df_sc_score = sc.createDataFrame(df_als_score)
df_sc_score.show()

+---------------+---------+-----+
|        profile|anime_uid|score|
+---------------+---------+-----+
| DesolatePsyche|    34096|    8|
|      baekbeans|    34599|   10|
|           skrn|    28891|    7|
|   edgewalker00|     2904|    9|
|aManOfCulture99|     4181|   10|
|          eneri|     2904|   10|
| Waffle_Empress|    16664|    6|
|   NIGGER_BONER|     2904|    8|
|         jchang|     2904|   10|
|    shadowsplat|     4181|    4|
|   angelsreview|     4672|    8|
| CalebTheMenace|     4181|    9|
|        Kiethol|     4181|   10|
|          Eanki|    34599|    8|
|      NekoKyupi|    34599|    9|
|          12sed|     5114|   10|
|  OVERPOWERED99|    30276|    8|
|  MrAnimeCrunch|    30276|   10|
|    JoJo_Stalin|    30276|    7|
|        Kaishuu|     4181|   10|
+---------------+---------+-----+
only showing top 20 rows



In [85]:
indexer = StringIndexer(inputCol="profile", outputCol="profile_index")

transformed = indexer.fit(df_sc_score).transform(df_sc_score)
transformed.show()

+---------------+---------+-----+-------------+
|        profile|anime_uid|score|profile_index|
+---------------+---------+-----+-------------+
| DesolatePsyche|    34096|    8|         32.0|
|      baekbeans|    34599|   10|       1104.0|
|           skrn|    28891|    7|       1825.0|
|   edgewalker00|     2904|    9|       3796.0|
|aManOfCulture99|     4181|   10|       9589.0|
|          eneri|     2904|   10|       9872.0|
| Waffle_Empress|    16664|    6|        554.0|
|   NIGGER_BONER|     2904|    8|      13796.0|
|         jchang|     2904|   10|        653.0|
|    shadowsplat|     4181|    4|      45563.0|
|   angelsreview|     4672|    8|          8.0|
| CalebTheMenace|     4181|    9|       5402.0|
|        Kiethol|     4181|   10|       4353.0|
|          Eanki|    34599|    8|        918.0|
|      NekoKyupi|    34599|    9|      13855.0|
|          12sed|     5114|   10|       3999.0|
|  OVERPOWERED99|    30276|    8|         84.0|
|  MrAnimeCrunch|    30276|   10|      2

25/05/28 11:57:46 WARN DAGScheduler: Broadcasting large task binary with size 1979.0 KiB


In [87]:
(training,test)=transformed.randomSplit([0.8, 0.2])

In [89]:
# There's a mean of 2.72 score reviews by profile, so we decide to use
# rank = 3
# maxIter = 10
# regParam = 0.1
als = ALS(rank=2,
          userCol="profile_index", itemCol="anime_uid",
          ratingCol="score", coldStartStrategy="drop", nonnegative=True)

model = als.fit(training)

25/05/28 11:58:12 WARN DAGScheduler: Broadcasting large task binary with size 2007.8 KiB
25/05/28 11:58:12 WARN DAGScheduler: Broadcasting large task binary with size 2010.2 KiB
25/05/28 11:58:13 WARN DAGScheduler: Broadcasting large task binary with size 2011.7 KiB
25/05/28 11:58:13 WARN DAGScheduler: Broadcasting large task binary with size 2013.0 KiB
25/05/28 11:58:13 WARN DAGScheduler: Broadcasting large task binary with size 2011.9 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2013.2 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2014.0 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2017.1 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2018.5 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2019.9 KiB
25/05/28 11:58:14 WARN DAGScheduler: Broadcasting large task binary with size 2021.3 KiB
25/05/28 11:58:14 WAR

In [91]:
evaluator = RegressionEvaluator(metricName="rmse",labelCol="score",predictionCol="prediction")

predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

print("RMSE="+str(rmse))

predictions.show()

25/05/28 11:58:19 WARN DAGScheduler: Broadcasting large task binary with size 1990.2 KiB
25/05/28 11:58:19 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/05/28 11:58:19 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/05/28 11:58:19 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/05/28 11:58:20 WARN DAGScheduler: Broadcasting large task binary with size 1990.3 KiB
25/05/28 11:58:20 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


RMSE=2.5592660458371874


25/05/28 11:58:20 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB


+----------------+---------+-----+-------------+----------+
|         profile|anime_uid|score|profile_index|prediction|
+----------------+---------+-----+-------------+----------+
| ZeroTheUltimate|     1088|   10|      15835.0|  8.064406|
|Angel_Apocalypse|    25517|    8|        710.0|  9.111655|
|          Satire|    25517|    7|        338.0| 6.9394593|
|     SupremeLord|    25517|    8|         33.0| 7.2374926|
|           kajia|     1088|    8|         14.0|  7.077851|
|     barcaman101|     1088|   10|        118.0|  8.236349|
|earl_of_sandvich|     6336|   10|         65.0|  9.274968|
|        Norayuki|    30654|    8|       2342.0| 7.0106564|
| LackOfARealLife|    30654|   10|        333.0|    8.6119|
|           Kuta1|    30654|   10|        355.0| 7.4910936|
|          Kyzari|    30654|    9|      13226.0|  8.193876|
|        Kenyanke|    17389|    8|       4352.0| 3.3377738|
|    SovietPsycho|    30654|    8|        339.0| 8.8195095|
|        Ijsberry|    30654|   10|      

In [93]:
recs=model.recommendForAllUsers(5).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['profile_index'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna() 
nrecs=nrecs.sort_values('profile_index')
nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['profile_index']], axis = 1)
nrecs.columns = [        
        'anime_uid',
        'score',
        'UserID_index'      
     ]
md=transformed.select(transformed['profile'],transformed['profile_index'],transformed['anime_uid'])
md=md.toPandas()
dict1 =dict(zip(md['profile_index'],md['profile']))
nrecs['profile']=nrecs['UserID_index'].map(dict1)
nrecs=nrecs.sort_values('profile')
nrecs.reset_index(drop=True, inplace=True)
new=nrecs[['profile','anime_uid','score']]
new['recommendations'] = list(zip(new.anime_uid, new.score))
res=new[['profile','recommendations']]  
df_score_reviews_based_reco=res['recommendations'].groupby([res.profile]).apply(list).reset_index()
print(df_score_reviews_based_reco.head())

25/05/28 11:58:35 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/05/28 11:58:39 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/05/28 11:58:46 WARN DAGScheduler: Broadcasting large task binary with size 1978.5 KiB


            profile                                    recommendations
0  -----noname-----  [(5172.0, 10.346378326416016), (19219.0, 10.58...
1   ---SnowFlake---  [(5172.0, 16.226808547973633), (19219.0, 16.50...
2       ---was-----  [(30030.0, 24.334794998168945), (18137.0, 24.2...
3      --EYEPATCH--  [(19219.0, 25.72243309020996), (39415.0, 25.40...
4          --Mizu--  [(2748.0, 26.11934471130371), (4286.0, 27.5448...


In [95]:
df_score_reviews_based_reco.to_csv("../data/als_score_reviews_based_reco.csv", index=False)